# Setup

In [1]:
import os
import glob
import cv2

# from pylibdmtx.pylibdmtx import decode
from ultralytics import YOLO, settings
root_dir = os.getcwd().replace('\\notebooks', '')
settings.update({'datasets_dir': f'{root_dir}/data/processed/test', 'runs_dir': f'{root_dir}/yolo/runs'})

# comment/uncomment to enable/disable printing
# print_amount = float('inf')
print_amount = 1

# Evaluation Functions

In [2]:
def eval_map(model, eval_yaml):
    '''
    Runs ultralytics val function to get mAP score and other metrics
    '''
    metrics = model.val(data=eval_yaml, split='test')

    print()
    print(f'Precision : {round(metrics.box.p[0], 2)}')  # close to 1 is good
    print(f'Recall    : {round(metrics.box.r[0], 2)}')  # close to 1 is good
    print(f'F1        : {round(metrics.box.f1[0], 2)}') # close to 1 is good
    print(f'mAP50-95  : {round(metrics.box.map, 3)}')   # 0.3 can be good, higher is better

    return

In [3]:
def crop_decoding(model, glob_path):
    '''
    Performs cropping and decoding of the images
    '''
    img_paths = glob.glob(glob_path)

    # stat tracking
    num_decodings = 0
    num_valid_decodings = 0

    idx = 0
    for img_path in img_paths:
        img = cv2.imread(img_path)
        results = model(img)
        boxes = results[0].boxes.xyxy.tolist()

        if boxes is not None:
            decoding = False
            valid_decoding = False
            for box in boxes:
                crop_obj = img[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
                # decodings = decode(crop_obj) # uncomment to decode
                # if decodings is not None:
                #     decoding = True
                #     actual_decoding = os.path.basename(img_path).split('_')[0]
                #     for decoding in decodings:
                #         if decoding.data == actual_decoding:
                #             valid_decoding = True
                #             break

                # optional saving
                cv2.imwrite(f'../data/cropped/{idx}.jpg', crop_obj)
                idx += 1
            
            if decoding is True:
                num_decodings += 1
                if valid_decoding is True:
                    num_valid_decodings += 1

    # calculate stats
    print(f'Dm decode rate: {num_decodings/len(img_paths)}')
    if num_decodings != 0:
        print(f'Valid decode rate: {num_valid_decodings/num_decodings}')
    else:
        print(f'Valid decode rate: 0 (no decodings)')
    return

In [4]:
def eval_yolo(model_path, glob_path, eval_yaml, print_count=1):
    '''
    Given a yolo model, makes the following evaluations:
     - mAP scores for bounding boxes
     - DM decode rate (% of decodings of test images)
     - Valid DM decode rate (% of decodings of test images that match with the serial number)
    Also prints the first "print_count" images with predictions.
    '''
    # Load the model
    model = YOLO(model_path)

    # mAP scores (saves to runs dir)
    eval_map(model, eval_yaml)

    # Crop and decode
    crop_decoding(model, glob_path)

    return

## Kaggle from Scratch

In [5]:
eval_yolo('../yolo/models/kaggle_scratch.pt', '../data/MAN/images/test/*.jpg', eval_yaml='../data/MAN/data.yaml', print_count=print_amount)

Ultralytics 8.3.6  Python-3.12.5 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


val: Scanning C:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\data\MAN\labels\test.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.48s/it]


                   all         50         50      0.256       0.24      0.106      0.069
Speed: 3.1ms preprocess, 7.6ms inference, 0.0ms loss, 5.4ms postprocess per image
Results saved to c:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\yolo\runs\detect\val

Precision : 0.26
Recall    : 0.24
F1        : 0.25
mAP50-95  : 0.069

0: 640x640 6 Data Matrixs, 21.0ms
Speed: 3.0ms preprocess, 21.0ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 19.0ms
Speed: 3.0ms preprocess, 19.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 19.0ms
Speed: 3.0ms preprocess, 19.0ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 22.0ms
Speed: 3.0ms preprocess, 22.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 19.0ms
Speed: 3.0ms preprocess, 19.0ms inference, 2.0ms post

## Kaggle finetuned

In [6]:
eval_yolo('../yolo/models/kaggle_finetuned.pt', '../data/MAN/images/test/*.jpg', eval_yaml='../data/MAN/data.yaml', print_count=print_amount)

Ultralytics 8.3.6  Python-3.12.5 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


val: Scanning C:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\data\MAN\labels\test.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.94s/it]


                   all         50         50      0.894      0.841      0.914      0.747
Speed: 4.1ms preprocess, 8.9ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to c:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\yolo\runs\detect\val2

Precision : 0.89
Recall    : 0.84
F1        : 0.87
mAP50-95  : 0.747

0: 640x640 1 Data Matrix, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 9.0ms
Speed: 3.0ms preprocess, 9.0ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 13.0ms
Speed: 3.0ms preprocess, 13.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 11.0ms
Speed: 3.0ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 10.0ms
Speed: 3.0ms preprocess, 10.0ms inference, 2.0ms postproces

## Ultralytics finetuned

In [7]:
eval_yolo('../yolo/models/ultralytics_finetuned.pt', '../data/MAN/images/test/*.jpg', eval_yaml='../data/MAN/data.yaml', print_count=print_amount)

Ultralytics 8.3.6  Python-3.12.5 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


val: Scanning C:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\data\MAN\labels\test.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.95s/it]


                   all         50         50      0.956        0.9      0.947      0.758
Speed: 3.4ms preprocess, 7.5ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to c:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\yolo\runs\detect\val3

Precision : 0.96
Recall    : 0.9
F1        : 0.93
mAP50-95  : 0.758

0: 640x640 1 Data Matrix, 8.0ms
Speed: 3.3ms preprocess, 8.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 15.0ms
Speed: 3.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 8.0ms
Speed: 3.0ms preprocess, 8.0ms inference, 2.0ms postprocess per